In [1]:
begin
	using Pkg
	Pkg.activate("..")
	Pkg.add("Cthulhu")
	# Pkg.instantiate()
	
	using PlutoUI
	using CairoMakie
	using LinearAlgebra
	using Luxor
	using BenchmarkTools
	using Printf
	using ImageMorphology
	using LoopVectorization
	using CUDA
	using CUDA:i32
	using Test
	using Cthulhu
end

  Activating project at `c:\Users\wenbl13\Desktop\project-distance-transforms-wenbo-2`
    Updating registry at `C:\Users\wenbl13\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\wenbl13\Desktop\project-distance-transforms-wenbo-2\Project.toml`
  No Changes to `C:\Users\wenbl13\Desktop\project-distance-transforms-wenbo-2\Manifest.toml`


In [2]:
boolean_indicator(f) = @. ifelse(f == 0, 1f10, 0f0)

boolean_indicator (generic function with 1 method)

In [3]:
function boolean_indicator(img::BitArray)
	f = similar(img, Float32)
	@turbo for i in CartesianIndices(f)
           @inbounds f[i] = img[i] ? 0f0 : 1f10
    end
	return f
end

boolean_indicator (generic function with 2 methods)

## Multi-Thded and Nonmulti-Thded

## 1D

In [4]:
begin
	function DT1helperA!(f, j)
		# i == -1 && j != -1
		temp=1
		while j>0
			@inbounds f[j]=temp^2
			j-=1
			temp+=1
		end
	end
	function DT1helperB!(f, i)
		# i != -1 && j == -1
		temp=1
		l = length(f)
		while i<=l
			@inbounds f[i]=temp^2
			i+=1
			temp+=1
		end
	end
	function DT1helperC!(f, i, j)
		# i != -1 && j != -1
		temp=1
		while(i<=j)
			@inbounds f[i]=f[j]=temp^2
			temp+=1
			i+=1
			j-=1
		end
	end 
	function DT1Wenbo!(f)
		pointerA = 1
		l = length(f)
		while pointerA <= l
			while pointerA <= l && @inbounds f[pointerA] == 0
				pointerA+=1
			end
			pointerB = pointerA
			while pointerB <= l && @inbounds f[pointerB] == 1f10
				pointerB+=1
			end
			if pointerB > length(f)
				if pointerA == 1
					return
				else
					DT1helperB!(f, pointerA)
				end
			else
				if pointerA == 1
					DT1helperA!(f, pointerB-1)
				else
					DT1helperC!(f, pointerA, pointerB-1)
				end
			end
			pointerA=pointerB
		end
	end
	function DT1Wenbo(f)
		f = boolean_indicator(f)
		DT1Wenbo!(f)
		return f
	end
end

DT1Wenbo (generic function with 1 method)

## 2D

In [5]:
begin
	function encode(leftD, rightf)
		if rightf == 1f10
			return -leftD
		end
		idx = 0
		while(rightf>1)
			rightf  /=10
			idx+=1 
		end
		return -leftD-idx/10-rightf/10
	end
	function decode(curr)	
		curr *= -10   				
		temp = Int(floor(curr))		
		curr -= temp 				
		if curr == 0
			return 1f10
		end
		temp %= 10
		while temp > 0
			temp -= 1
			curr*=10
		end
		return round(curr)
	end
	function DT2Helper!(f)
		l = length(f)
		pointerA = 1
		while pointerA<=l && @inbounds f[pointerA] <= 1
			pointerA += 1
		end
		p = 0
		while pointerA<=l
			@inbounds curr = f[pointerA]
			prev = curr
			temp = min(pointerA-1, p+1)
			p = 0
			while (0 < temp)
				@inbounds fi = f[pointerA-temp]
				fi = fi < 0 ? decode(fi) : fi
				newDistance = muladd(temp, temp, fi)
				if newDistance < curr
					curr = newDistance
					p = temp
				end
				temp -= 1
			end
			temp = 1
			templ = length(f) - pointerA
			while (temp <= templ && muladd(temp, temp, -curr) < 0)
				@inbounds curr = min(curr, muladd(temp, temp, f[pointerA+temp]))
				temp += 1
			end
			@inbounds f[pointerA] = encode(curr, prev)
			# end
			pointerA+=1
			while pointerA<=l && @inbounds f[pointerA] <= 1
				pointerA += 1
			end
		end
		i = 0
		while i<l
			i+=1
			f[i] = floor(abs(f[i]))
		end
	end
	function DT2WenboA!(f)
		for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2WenboB!(f)
		Threads.@threads for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		Threads.@threads for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2Wenbo(f)
		f = boolean_indicator(f)
		DT2tf! = length(f) > 2700 && Threads.nthreads()>1 ?  DT2WenboB! : DT2WenboA!
		DT2tf!(f)
		return f
	end
end

DT2Wenbo (generic function with 1 method)

## 3D

In [6]:
begin
	function DT3WenboA!(f)
		for i in axes(f, 3)
		    @inbounds DT2WenboA!(@view(f[:, :, i]))
		end
		for i in CartesianIndices(f[:,:,1])
			@inbounds DT2Helper!(@view(f[i, :]))
		end
	end 
	function DT3WenboB!(f)
		Threads.@threads for i in axes(f, 3)
		    @inbounds DT2WenboB!(@view(f[:, :, i]))
		end
		Threads.@threads for i in CartesianIndices(f[:,:,1])
			@inbounds DT2Helper!(@view(f[i, :]))
		end
	end
	function DT3Wenbo(f)
		f = boolean_indicator(f)
		DT3tf! = length(f) > 2700 && Threads.nthreads()>1 ?  DT3WenboB! : DT3WenboA!
		DT3tf!(f)
		return f
	end
end

DT3Wenbo (generic function with 1 method)

## Timing: Wenbo vs ImageMorphology

In [5]:
euclideanImageMorphology(img::BitArray) = distance_transform(feature_transform(img))

euclideanImageMorphology (generic function with 1 method)

In [6]:
begin
	img1DsmallB = Bool.(rand([0, 1], 15))
	img1DbigB = Bool.(rand([0, 1], 4000))
	img1DB = Bool.(rand([0, 1], 200))
	img2DB = Bool.(rand([0, 1], 200, 800))
	img2DsmallB = Bool.(rand([0, 1], 7, 7))
	img3DB = Bool.(rand([0, 1], 200, 400, 600))
	img2D4kB = Bool.(rand([0, 1], 3840, 2160))
	img1DsmallG = CuArray(img1DsmallB)
	img1DbigG = CuArray(img1DbigB)
	img1DG = CuArray(img1DB)
	img2DG = CuArray(img2DB)
	img2DsmallG = CuArray(img2DsmallB)
	img3DG = CuArray(img3DB)
	img2D4kG = CuArray(img2D4kB)
	"Test inputs created."
end

"Test inputs created."

### 1D

In [9]:
let
	n = 200
	for i = 1: 100
		f = Bool.(rand([0, 1], n))
		rslt1 = euclideanImageMorphology(f);
		rslt1 .^ 2
		rslt2 = DT1Wenbo(f)
		for i in CartesianIndices(rslt1)
			if rslt1[i] - rslt2[i] != 0.0
				"failed"
				break
			end
		end
	end
	"1D: 100 test cases passed!"
end

"1D: 100 test cases passed!"

In [10]:
@benchmark DT1Wenbo($img1DB)

BenchmarkTools.Trial: 10000 samples with 401 evaluations.
 Range (min … max):  235.411 ns … 30.528 μs  ┊ GC (min … max):  0.00% … 98.27%
 Time  (median):     457.606 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   488.570 ns ±  1.298 μs  ┊ GC (mean ± σ):  13.21% ±  4.91%

     ▃▄                 ▂▂▂▂▅█                                  
  ▃▇▇██▇▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▄██████▇▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  235 ns          Histogram: frequency by time          777 ns <

 Memory estimate: 896 bytes, allocs estimate: 1.

In [11]:
@benchmark euclideanImageMorphology($img1DB)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.340 μs …  1.163 ms  ┊ GC (min … max):  0.00% … 99.71%
 Time  (median):     2.430 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.657 μs ± 25.790 μs  ┊ GC (mean ± σ):  17.17% ±  2.44%

  ▆█▅▂ ▃▆▇▅▂▁                 ▂▃▂▁▁▂▃▂▁                      ▂
  ███████████████▇▇▇▇▇▆▅▅▅▃▃▅████████████▇▇▆▇▇▆▆▆▅▅▆▆▅▆▅▅▅▅▅ █
  1.34 μs      Histogram: log(frequency) by time     11.5 μs <

 Memory estimate: 8.97 KiB, allocs estimate: 6.

### 2D

In [12]:
let
	n = 200
	for i = 1: 100
		f = Bool.(rand([0, 1], n, n))
		rslt1 = euclideanImageMorphology(f);
		rslt1 .^ 2
		rslt2 = DT2Wenbo(f)
		for i in CartesianIndices(rslt1)
			if rslt1[i] - rslt2[i] != 0.
				"failed"
				break
			end
		end
	end
	"2D: 100 test cases passed!"
end

"2D: 100 test cases passed!"

In [13]:
@benchmark DT2Wenbo($img2DB)

BenchmarkTools.Trial: 3221 samples with 1 evaluation.
 Range (min … max):  1.236 ms …  14.492 ms  ┊ GC (min … max): 0.00% … 89.20%
 Time  (median):     1.505 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.549 ms ± 580.793 μs  ┊ GC (mean ± σ):  2.41% ±  5.58%

                      ▃ ▂▆█▇▅▂                                 
  ▁▁▁▁▁▁▁▁▁▂▂▃▃▃▃▃▃▄▆██████████▇▆▄▃▂▃▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.24 ms         Histogram: frequency by time        1.88 ms <

 Memory estimate: 625.05 KiB, allocs estimate: 2.

In [14]:
@benchmark euclideanImageMorphology($img2DB)

BenchmarkTools.Trial: 938 samples with 1 evaluation.
 Range (min … max):  4.411 ms … 19.059 ms  ┊ GC (min … max): 0.00% … 58.78%
 Time  (median):     5.033 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.322 ms ±  1.490 ms  ┊ GC (mean ± σ):  4.42% ± 10.23%

   ▅███▆▄▂                                                    
  ▇████████▅▄▁▄▄▆▄▄▁▄▆▅▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▅▆▆▁▇ █
  4.41 ms      Histogram: log(frequency) by time     13.8 ms <

 Memory estimate: 3.75 MiB, allocs estimate: 10.

### 3D

In [15]:
let
	n = 200
	for i = 1: 100
		f = Bool.(rand([0, 1], n, n, n))
		rslt1 = euclideanImageMorphology(f);
		rslt1 .^ 2
		rslt2 = DT3Wenbo(f)
		for i in CartesianIndices(rslt1)
			if rslt1[i] - rslt2[i] != 0.0
				"failed"
				break
			end
		end
	end
	"3D: 100 test cases passed!"
end

"3D: 100 test cases passed!"

In [16]:
@benchmark DT3Wenbo($img3DB)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  552.447 ms … 755.521 ms  ┊ GC (min … max): 0.00% … 27.79%
 Time  (median):     577.809 ms               ┊ GC (median):    0.12%
 Time  (mean ± σ):   595.573 ms ±  62.847 ms  ┊ GC (mean ± σ):  5.41% ±  9.19%

  █  ▁▁  ▁   ▁▁   ▁                                           ▁  
  █▁▁██▁▁█▁▁▁██▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  552 ms           Histogram: frequency by time          756 ms <

 Memory estimate: 183.41 MiB, allocs estimate: 4.

In [17]:
@benchmark euclideanImageMorphology($img3DB)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.493 s …   2.629 s  ┊ GC (min … max): 0.13% … 4.41%
 Time  (median):     2.561 s              ┊ GC (median):    2.33%
 Time  (mean ± σ):   2.561 s ± 96.022 ms  ┊ GC (mean ± σ):  2.33% ± 3.03%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.49 s         Histogram: frequency by time        2.63 s <

 Memory estimate: 1.44 GiB, allocs estimate: 480009.

# Rewriting Wenbo!() for GPU Compatibility

In [7]:
function _boolean_indicator_GPU!(out, f)
	i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
	@inbounds out[i] = f[i] ? 0f0 : 1f10
	return
end

_boolean_indicator_GPU! (generic function with 1 method)

In [8]:
begin
	b_i_GPU_kernels = []
	kernel = @cuda launch=false _boolean_indicator_GPU!(CuArray{Float32, 1}(undef, 1), CuArray{Bool, 1}(undef, 1))
	push!(b_i_GPU_kernels, kernel)
	kernel = @cuda launch=false _boolean_indicator_GPU!(CuArray{Float32, 2}(undef, 1, 1), CuArray{Bool, 2}(undef, 1, 1))
	push!(b_i_GPU_kernels, kernel)
	kernel = @cuda launch=false _boolean_indicator_GPU!(CuArray{Float32, 3}(undef, 1, 1, 1), CuArray{Bool, 3}(undef, 1, 1, 1))
	push!(b_i_GPU_kernels, kernel)
	config_threads = launch_configuration(kernel.fun).threads
	"Created boolean_indicator_GPU kernels."
end

"Created boolean_indicator_GPU kernels."

In [9]:
function boolean_indicator_GPU(f)
		# input = CuArray(img)
		output = similar(f, Float32)
		threads = min(length(f), config_threads)
		blocks = cld(length(f), threads)
		b_i_GPU_kernels[ndims(f)](output, f; threads, blocks)
		return output
end

boolean_indicator_GPU (generic function with 1 method)

In [9]:
@benchmark boolean_indicator($img2DB)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   36.000 μs …  11.151 ms  ┊ GC (min … max):  0.00% … 98.27%
 Time  (median):     162.200 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   182.997 μs ± 467.589 μs  ┊ GC (mean ± σ):  16.70% ±  6.52%

  ▇▃▃▂           ▂▅▇▆█▆▅▄▃▃▃▂▂▂▂▂▁▁▁ ▁                          ▂
  █████▇▇▅▅▃▄▄▄▃▅████████████████████████▆▆▆▆▆▆▅▅▄▅▅▆▆▆▅▆▅▅▇▆▅▆ █
  36 μs         Histogram: log(frequency) by time        433 μs <

 Memory estimate: 625.05 KiB, allocs estimate: 2.

In [85]:
begin
	function leftEnd!(out,f)
		i = 1
		while i <= length(f) && @inbounds !f[i]
			i += 1
		end
		out[i] = 0f0
		i -= 1
		ct = 1
		while i >= 0
			@inbounds out[i] = ct^2
			ct += 1
			i -= 1
		end
		return
	end
	kernel1 = @cuda launch=false leftEnd!(CuArray{Float32, 1}(undef, 1), CuArray{Bool, 1}(undef, 0))

	function rightEnd!(out,f)
		i = length(f)
		while i >= 0 && @inbounds !f[i]
			i -= 1
		end
		i += 1
		ct = 1
		while i <= length(f)
			@inbounds out[i] = ct^2
			ct += 1
			i += 1
		end
		return
	end
	kernel2 = @cuda launch=false rightEnd!(CuArray{Float32, 1}(undef, 1), CuArray{Bool, 1}(undef, 0))

	function temp2!(out, f, ct)
		i = threadIdx().x
		block_start = (i-1)*ct+1
		block_end = i*ct
		prev= -1
		j = block_start
		while j <= block_end
			if @inbounds f[j]
				prev = j
				break
			end
			j += 1
		end
		if prev == -1
			return
		end
		j = prev+1
		while j <= block_end
			if @inbounds f[j]
				ct = 1
				out[j] = 0f0
				while prev+2*ct-j <= 0
					@inbounds out[j-ct] = out[prev+ct] = ct^2
					ct += 1
				end
				prev = j
			end
			j += 1
		end
		while j <= length(f)
			if @inbounds f[j]
				ct = 1
				out[j] = 0f0
				while prev+2*ct-j <= 0
					@inbounds out[j-ct] = out[prev+ct] = ct^2
					ct += 1
				end
				return
			end
			j += 1
		end
		return
	end

	kernel3 = @cuda launch=false temp2!(CuArray{Float32, 1}(undef, 0), CuArray{Bool, 1}(undef, 0),0)
	function DT1Wenbo_GPU(f)
		l = length(f)
		output = CuArray{Float32, 1}(undef, l)
		ct = cld(l, config_threads)
		threads = fld(l,ct)
		kernel3(output, f, ct; threads)
		kernel1(output, f)
		kernel2(output, f)
		# f = boolean_indicator_GPU(f)
		# DT1Wenbo_GPU!(f)
		return output
	end
end

DT1Wenbo_GPU (generic function with 1 method)

In [86]:
begin
	function DT1Wenbo_GPU!(out, f)
		pointerA = 1
		l = length(f)
		while pointerA <= l
			while pointerA <= l && @inbounds f[pointerA]
				out[pointerA] = 0f0
				pointerA+=1
			end
			pointerB = pointerA
			while pointerB <= l && @inbounds !f[pointerB]
				pointerB+=1
			end
			if pointerB > length(f)
				if pointerA == 1
					i=length(f)
					while i>0
						@inbounds out[i]=1f10
						i-=1
					end
				else
					i = pointerA 
					out[pointerA-1] = 0f0
					temp=1
					l = length(f)
					while i<=l
						@inbounds out[i]=temp^2
						i+=1
						temp+=1
					end
				end
			else
				if pointerA == 1
					j = pointerB-1
					out[pointerB] = 0f0
					temp=1
					while j>0
						@inbounds out[j]=temp^2
						j-=1
						temp+=1
					end
				else
					out[pointerA-1] = 0f0
					out[pointerB] = 0f0
					i = pointerA
					j = pointerB-1
					temp=1
					while i<=j
						@inbounds out[i]=out[j]=temp^2
						temp+=1
						i+=1
						j-=1
					end
				end
			end
			pointerA=pointerB
		end
	end

	kernel4 = @cuda launch=false DT1Wenbo_GPU!(CuArray{Float32, 1}(undef, 0), CuArray{Bool, 1}(undef, 0))

	function DT1Wenbo_GPU2(f)
		output = CuArray{Float32, 1}(undef, length(f))
		kernel4(output, f)
		return output
	end
end

DT1Wenbo_GPU2 (generic function with 1 method)

In [93]:
@benchmark DT1Wenbo($img1DB)

BenchmarkTools.Trial: 10000 samples with 386 evaluations.
 Range (min … max):  236.269 ns … 48.417 μs  ┊ GC (min … max):  0.00% … 98.88%
 Time  (median):     448.187 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   507.644 ns ±  1.696 μs  ┊ GC (mean ± σ):  14.50% ±  4.30%

   ▂                  █▃▃▁                                      
  ▄█▇▆▃▂▂▁▁▁▁▁▁▁▁▁▁▃▄▅█████▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  236 ns          Histogram: frequency by time          824 ns <

 Memory estimate: 896 bytes, allocs estimate: 1.

In [88]:
@benchmark DT1Wenbo_GPU($img1DG)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.650 μs …  1.843 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.050 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.301 μs ± 18.713 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▆▆█▇▇▆▅▄▃▄▃▃▃▂▁▁▂▁▂▂▂▁▁                                   ▂
  ██████████████████████████▇██▇███▇▇▇▇▇▇▇▇▆▇▇█▆▇▇▅▆▅▃▆▅▆▁▅▅ █
  5.65 μs      Histogram: log(frequency) by time     23.3 μs <

 Memory estimate: 2.66 KiB, allocs estimate: 58.

In [87]:
@benchmark DT1Wenbo_GPU2($img1DG) 

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):   2.100 μs …   2.084 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      2.943 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.112 μs ± 196.377 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                             
  █▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▂▂▂▁▂▂ ▂
  2.1 μs          Histogram: frequency by time         1.68 ms <

 Memory estimate: 1008 bytes, allocs estimate: 22.

In [134]:
CUDA.reclaim()

In [30]:
rslt_gpu = Array(DT1Wenbo_GPU2(img1DbigG))

4000-element Vector{Float32}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0

In [26]:
rslt_cpu = DT1Wenbo(img1DbigB)

4000-element Vector{Float32}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0

In [28]:
n = 7

for i = n-5: n+5
    print("$(i)\t")
end
println()

for i = n-5: n+5
    print("$(img1DbigB[i])\t")
end
println()

for i = n-5: n+5
    print("$(rslt_cpu[i])\t")
end
println()

for i = n-5: n+5
    print("$(rslt_gpu[i])\t")
end

2	3	4	5	6	7	8	9	10	11	12	
true	false	true	true	true	true	true	false	true	false	true	
0.0	1.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	1.0	0.0	
0.0	1.0	0.0	1.0	0.0	2.3694275e-38	0.0	1.0	0.0	1.0	0.0	

In [31]:
for i = 1: length(img1DG)
    if rslt_cpu[i] != rslt_gpu[i]
        println("i = $i, cpu: $(rslt_cpu[i]), gpu: $(rslt_gpu[i])")
    end
end

In [32]:
@test Array(DT1Wenbo_GPU2(img1DbigG)) == DT1Wenbo(img1DbigB)

Test Passed
  Expression: Array(DT1Wenbo_GPU2(img1DbigG)) == DT1Wenbo(img1DbigB)
   Evaluated: Float32[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0  …  0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0] == Float32[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0  …  0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]

In [77]:
begin
	function encode_GPU!(out, leftD, rightf)
		out[1] = -leftD
		if rightf == 1f10
			return nothing
		end
		idx = 0
		while(rightf>1)
			rightf  /=10
			idx+=1 
		end
        out[1] += -idx/10-rightf/10
		return nothing
	end
	encode_kernel = @cuda launch=false encode_GPU!(CuArray{Float32, 1}(undef, 1),0f0,0f0)

	function decode_GPU!(out, curr)	
		curr *= -10   				
		temp = Int(floor(curr))		
		curr -= temp 				
		if curr == 0
			out[1] = 1f10
			return nothing
		end
		temp %= 10
		while temp > 0
			temp -= 1
			curr*=10
		end
		out[1] = round(curr)
		return nothing
	end
	decode_kernel = @cuda launch=false decode_GPU!(CuArray{Float32, 1}(undef, 1),0f0)

	function DT2Helper_GPU!(f)
		l = length(f)
		pointerA = 1
		while pointerA<=l && @inbounds f[pointerA] <= 1
			pointerA += 1
		end
		p = 0
		while pointerA<=l
			@inbounds curr = f[pointerA]
			prev = curr
			temp = min(pointerA-1, p+1)
			p = 0
			while (0 < temp)
				@inbounds fi = f[pointerA-temp]
				fi = fi < 0 ? decode(fi) : fi
				newDistance = muladd(temp, temp, fi)
				if newDistance < curr
					curr = newDistance
					p = temp
				end
				temp -= 1
			end
			temp = 1
			templ = length(f) - pointerA
			while (temp <= templ && muladd(temp, temp, -curr) < 0)
				@inbounds curr = min(curr, muladd(temp, temp, f[pointerA+temp]))
				temp += 1
			end
			@inbounds f[pointerA] = encode(curr, prev)
			# end
			pointerA+=1
			while pointerA<=l && @inbounds f[pointerA] <= 1
				pointerA += 1
			end
		end
		i = 0
		while i<l
			i+=1
			f[i] = floor(abs(f[i]))
		end
	end
	function DT2WenboA!(f)
		for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2WenboB!(f)
		Threads.@threads for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		Threads.@threads for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2Wenbo(f)
		f = boolean_indicator(f)
		DT2tf! = length(f) > 2700 && Threads.nthreads()>1 ?  DT2WenboB! : DT2WenboA!
		DT2tf!(f)
		return f
	end
end

DT2Wenbo (generic function with 1 method)

In [85]:
encode_GPU(13,25)

-13.224999f0

In [88]:
@benchmark decode(-13.224999f0)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.900 ns … 30.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.000 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.090 ns ±  0.992 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄       █       ▅       ▂                                  ▁
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█ █
  0.9 ns       Histogram: log(frequency) by time      1.6 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [86]:
@benchmark decode_kernel(temptemp,-13.224999f0)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.360 μs …  1.445 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.710 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.272 μs ± 22.292 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅▇█▇▆▆▅▇█▇▇▇▅▄▃▂▂▁ ▁                                       ▃
  █████████████████████▇██▇▇█▇█▇▇▇▇▇▇▇▆▆▇▇██▇▇▇▇█▇▆▇▅▅▅▅▅▆▅▅ █
  1.36 μs      Histogram: log(frequency) by time     3.89 μs <

 Memory estimate: 720 bytes, allocs estimate: 18.

In [11]:
function k1!(in::CuArray{Float32, 1, CUDA.Mem.DeviceBuffer})::Nothing
    in[i] = Float32(1f0)
    return
end

k1! (generic function with 2 methods)

In [12]:
k1_kernel = @cuda launch=false k1!(temp, 0)

CUDA.HostKernel{typeof(k1!), Tuple{CuDeviceVector{Float32, 1}, Int64}}(k1!, CuFunction(Ptr{Nothing} @0x000000009a4b3010, CuModule(Ptr{Nothing} @0x000000009a201c00, CuContext(0x0000000080e6e790, instance e1dfd01fd5737dda))), CUDA.KernelState(Ptr{Nothing} @0x0000000904000000))

In [13]:
temp = CuArray([0f0,0f0,0f0])

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0
 0.0
 0.0

In [14]:
function k2!(in::CuArray{Float32, 1, CUDA.Mem.DeviceBuffer})::Nothing
    for i in length(in)
        if in[i] == 0f0
            @cuda k1!(in,i)
        end
    end
end

k2! (generic function with 2 methods)

In [16]:
@cuda k2!(temp)
# @descend k2!(temp)

GPUCompiler.InvalidIRError: InvalidIRError: compiling kernel #k2!(CuDeviceVector{Float32, 1}) resulted in invalid LLVM IR
Reason: unsupported call through a literal pointer (call to memcmp)
Stacktrace:
 [1] _memcmp
   @ .\strings\string.jl:109
 [2] ==
   @ .\strings\string.jl:121
 [3] macro expansion
   @ C:\Users\wenbl13\.julia\packages\CUDA\DfvRa\src\pool.jl:72
 [4] macro expansion
   @ C:\Users\wenbl13\.julia\packages\CUDA\DfvRa\lib\utils\memoization.jl:78
 [5] stream_ordered
   @ C:\Users\wenbl13\.julia\packages\CUDA\DfvRa\src\pool.jl:71
 [6] multiple call sites
   @ unknown:0
Reason: unsupported dynamic function invocation (call to get!(constructor, x::CUDA.APIUtils.LazyInitialized; hook) in CUDA.APIUtils at C:\Users\wenbl13\.julia\packages\CUDA\DfvRa\lib\utils\threading.jl:24)
Stacktrace:
 [1] device_contexts
   @ C:\Users\wenbl13\.julia\packages\CUDA\DfvRa\lib\cudadrv\state.jl:189
 [2] multiple call sites
   @ unknown:0
Reason: unsupported dynamic function invocation (call to id

In [72]:
CUDA.@device_code_warntype k2!(temp)

PTX CompilerJob of kernel #k1!(CuDeviceVector{Float32, 1}) for sm_80, always_inline=false

MethodInstance for k1!(::CuDeviceVector{Float32, 1})
  from k1!(in) in Main at c:\Users\wenbl13\Desktop\project-distance-transforms-wenbo-2\julia\dt_wenbo_final.ipynb:1
Arguments
  #self#::Core.Const(k1!)
  in::CuDeviceVector{Float32, 1}
Body::Nothing
1 ─     Base.setindex!(in, 1.0f0, 1)
└──     return Main.nothing



In [10]:
function k3!(out, f, l)
	i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    @inbounds if f[i]
        @inbounds out[i] = 0f0
        @cuprintln("out[$i] = $(out[i])")
        return
    end
    @cuprintln("$i here")
    ct = 1
    while i+ct <= l && i-ct > 0
        @inbounds if f[i+ct] || f[i-ct]
            @inbounds out[i] = ct*ct
            @cuprintln("out[$i] = $(out[i])")
            return
        end
        ct += 1
    end
    while i+ct <= l
        @inbounds if f[i+ct]
            @inbounds out[i] = ct*ct
            @cuprintln("out[$i] = $(out[i])")
            return
        end
        ct += 1
    end
    while i-ct > 0
        @inbounds if f[i-ct]
            @inbounds out[i] = ct*ct
            @cuprintln("out[$i] = $(out[i])")
            return
        end
        ct += 1
    end
    @inbounds out[i] = 1f10
    @cuprintln("out[$i] = $(out[i])")
    return
end
k3 = @cuda launch=false k3!(CuArray{Float32, 1}(undef, 0), CuArray{Bool, 1}(undef, 0),0)

CUDA.HostKernel{typeof(k3!), Tuple{CuDeviceVector{Float32, 1}, CuDeviceVector{Bool, 1}, Int64}}(k3!, CuFunction(Ptr{Nothing} @0x000000009e288a60, CuModule(Ptr{Nothing} @0x000000009e216d60, CuContext(0x00000000854c29c0, instance ddeffb5f5e88ab9))), CUDA.KernelState(Ptr{Nothing} @0x0000000903e00000))

In [35]:
function DT1Wenbo_GPU3(f)
    l = length(f)
    output = similar(f, Float32)
    threads = min(l, config_threads)
    blocks = cld(l, threads)
    k3(output, f, l; threads, blocks)
    return output
end

DT1Wenbo_GPU3 (generic function with 1 method)

In [37]:
img1DsmallG

15-element CuArray{Bool, 1, CUDA.Mem.DeviceBuffer}:
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

In [36]:
DT1Wenbo_GPU3(img1DsmallG)

15-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
  4.0
  1.0
  0.0
  0.0
  0.0
  1.0
  1.0
  0.0
  1.0
  4.0
  9.0
 16.0
 25.0
 36.0
 49.0

In [52]:
@benchmark DT1Wenbo(img1DbigB)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  4.867 μs … 806.033 μs  ┊ GC (min … max): 0.00% … 97.90%
 Time  (median):     5.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.678 μs ±  17.928 μs  ┊ GC (mean ± σ):  6.46% ±  2.40%

     ▃█▅▅▄                                                     
  ▃█▆█████▇▆▄▄▅▅▄▆▅▄▄▃▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  4.87 μs         Histogram: frequency by time        11.8 μs <

 Memory estimate: 15.75 KiB, allocs estimate: 1.

In [51]:
@benchmark DT1Wenbo_GPU3(img1DbigG)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):   2.433 μs …   1.917 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      3.244 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.134 μs ± 143.947 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                             
  █▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▁▁▂▁▁▂▂▂▁▁▂ ▂
  2.43 μs         Histogram: frequency by time         1.25 ms <

 Memory estimate: 1.19 KiB, allocs estimate: 32.

In [69]:
let 
    row_l=10
    for i = 1: 200
        row = cld(i, row_l)
        col = i - (row-1)*row_l
        println("[$row, $col]")
    end
end

[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[1, 10]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]
[2, 10]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[3, 9]
[3, 10]
[4, 1]
[4, 2]
[4, 3]
[4, 4]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[4, 9]
[4, 10]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[5, 6]
[5, 7]
[5, 8]
[5, 9]
[5, 10]
[6, 1]
[6, 2]
[6, 3]
[6, 4]
[6, 5]
[6, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[7, 1]
[7, 2]
[7, 3]
[7, 4]
[7, 5]
[7, 6]
[7, 7]
[7, 8]
[7, 9]
[7, 10]
[8, 1]
[8, 2]
[8, 3]
[8, 4]
[8, 5]
[8, 6]
[8, 7]
[8, 8]
[8, 9]
[8, 10]
[9, 1]
[9, 2]
[9, 3]
[9, 4]
[9, 5]
[9, 6]
[9, 7]
[9, 8]
[9, 9]
[9, 10]
[10, 1]
[10, 2]
[10, 3]
[10, 4]
[10, 5]
[10, 6]
[10, 7]
[10, 8]
[10, 9]
[10, 10]
[11, 1]
[11, 2]
[11, 3]
[11, 4]
[11, 5]
[11, 6]
[11, 7]
[11, 8]
[11, 9]
[11, 10]
[12, 1]
[12, 2]
[12, 3]
[12, 4]
[12, 5]
[12, 6]
[12, 7]
[12, 8]
[12, 9]
[12, 10]
[13, 1]
[13, 2]
[13, 3]
[13, 4]
[13, 5]
[13, 6]
[13, 7]
[13, 8]
[13, 9]
[13, 10]
[14, 1]
[14, 2]
[14, 3]
[14, 4]
[14, 

In [70]:
function k5!(out, f, row_l, l)
	i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i > l
        return
    end 
    row = cld(i, row_l)
    col = i - (row-1)*row_l
    out[row, col] = i
    # @inbounds if f[row, col]
    #     @inbounds out[row, col] = 0f0
    #     return nothing
    # end
    # ct = 1
    # # while col+ct <= row_l && col-ct > 0
    # #     @inbounds temp2 = f[row, col-ct]
    # #     @inbounds temp1 = f[row, col+ct]
    # #     if temp1 || temp2
    # #         @inbounds out[row, col] = ct*ct
    # #         return nothing
    # #     end
    # #     ct += 1
    # # end
    # @inbounds out[row, col] = 1f10
    
    # while col-ct > 0
    #     @inbounds if f[row, col-ct]
    #         @inbounds out[row, col] = ct*ct
    #         break
    #     end
    #     ct += 1    
    # end

    # ct = 1

    # while col+ct <= row_l
    #     @inbounds if f[row, col+ct]
    #         @inbounds out[row, col] = min(out[row, col], ct*ct)
    #         return nothing
    #     end
    #     ct += 1
    # end
    return nothing
end
k5 = @cuda launch=false k5!(CuArray{Float32, 2}(undef, 10, 10), CuArray{Bool, 2}(undef, 10, 10),5,0)

CUDA.HostKernel{typeof(k5!), Tuple{CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Bool, 1}, Int64, Int64}}(k5!, CuFunction(Ptr{Nothing} @0x00000000ced52810, CuModule(Ptr{Nothing} @0x000000007de51150, CuContext(0x00000000830884a0, instance 6cffdcba5ef62040))), CUDA.KernelState(Ptr{Nothing} @0x0000000903e00000))

In [19]:
function k4_copy_to_new_array!(org, out, row_l, col_l, l)
	i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    if i > l
        return
    end 
    row = fld(i, row_l)+1
    col = i%row_l+1
    # @cuprintln("[cld($i, $row_l), $i mod $row_l+1] = [$row, $col]")
    ct = 1
    @inbounds while ct < CUDA.sqrt(out[row, col]) && row+ct <= col_l
        @inbounds if org[row+ct, col] < out[row, col]
            @inbounds out[row, col] = min(out[row, col], muladd(ct,ct,org[row+ct, col]))
        end
        ct += 1
    end
    ct = 1
    @inbounds while ct < CUDA.sqrt(out[row, col]) && row-ct > 0
        # @cuprintln("[$row, $col] = $(out[row, col])  -> [$(row-ct), $col] = $(org[row-ct, col])")
        @inbounds if org[row-ct, col] < out[row, col]
            @inbounds out[row, col] = min(out[row, col], muladd(ct,ct,org[row-ct, col]))
        end
        ct += 1
    end
    # @cuprintln("[$row, $col] finished")
    return
end
k4 = @cuda launch=false k4_copy_to_new_array!(CuArray{Float32, 2}(undef, 0,0),CuArray{Float32, 2}(undef, 0,0),0,0,0)

CUDA.HostKernel{typeof(k4_copy_to_new_array!), Tuple{CuDeviceMatrix{Float32, 1}, CuDeviceMatrix{Float32, 1}, Int64, Int64, Int64}}(k4_copy_to_new_array!, CuFunction(Ptr{Nothing} @0x00000000cf377eb0, CuModule(Ptr{Nothing} @0x00000000ced2b3b0, CuContext(0x00000000830884a0, instance 6cffdcba5ef62040))), CUDA.KernelState(Ptr{Nothing} @0x0000000903e00000))

In [11]:
function DT2Wenbo_GPU2!(f)  
    row_length, col_length = size(f)
    l = length(f)
    output = similar(f, Float32)
    threads = min(l, config_threads)
    blocks = cld(l, threads)

    k5(output, f, row_length, l; threads, blocks)
    
    # output_copy , output,= deepcopy(output)
    # k4(output_copy row_length, col_length, l; threads, blocks)
    return output
end

DT2Wenbo_GPU2! (generic function with 1 method)

In [59]:
img2DsmallG

7×7 CuArray{Bool, 2, CUDA.Mem.DeviceBuffer}:
 1  1  0  1  1  0  1
 1  0  0  1  1  0  0
 0  1  1  1  0  0  0
 0  1  0  1  1  1  0
 1  1  1  0  1  0  0
 1  1  0  1  0  0  0
 0  1  0  0  1  0  0

In [13]:
DT2Wenbo(img2DsmallB)

┌ Warning: #= c:\Users\wenbl13\Desktop\project-distance-transforms-wenbo-2\julia\dt_wenbo_final.ipynb:3 =#:
│ `LoopVectorization.check_args` on your inputs failed; running fallback `@inbounds @fastmath` loop instead.
│ Use `warn_check_args=false`, e.g. `@turbo warn_check_args=false ...`, to disable this warning.
└ @ Main C:\Users\wenbl13\.julia\packages\LoopVectorization\gyra6\src\condense_loopset.jl:1037


7×7 Matrix{Float32}:
 0.0  0.0  0.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0
 1.0  2.0  1.0  0.0  0.0  0.0  1.0
 4.0  1.0  0.0  1.0  0.0  0.0  1.0
 4.0  1.0  0.0  0.0  0.0  0.0  0.0
 4.0  1.0  0.0  1.0  0.0  0.0  1.0

In [14]:
DT2Wenbo_GPU2!(img2DsmallG)

7×7 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0
 1.0  2.0  1.0  0.0  0.0  0.0  1.0
 4.0  1.0  0.0  1.0  0.0  0.0  1.0
 4.0  1.0  0.0  0.0  0.0  0.0  0.0
 4.0  1.0  0.0  1.0  0.0  0.0  1.0

In [15]:
DT2Wenbo(img2DB) == Array(DT2Wenbo_GPU2!(img2DG))

false

In [40]:
test_2d = Bool.(rand([0, 1], 200, 200))
test_2d_G = CuArray(test_2d);

In [74]:
rslt1 = DT2Wenbo(test_2d);

In [75]:
rslt2 = Array(DT2Wenbo_GPU2!(test_2d_G));

In [14]:
for i in CartesianIndices(rslt1)
    if rslt1[i] != rslt2[i]
        println(i)
        break
    end
end
println(".")

CartesianIndex(4, 1)
.


In [164]:
let 
    x = 50
    y = 5
    a = size(img2DB,1)*(x-1)+y
    println(a % 768)
end


In [76]:
x = 50
y = 5
function print_error(rslt, x, y)
    offset = 5
    print("\t\t")
    for j = max(y-offset,1): min(y+offset,size(rslt,2))
        print(j)
        print("\t")
    end
    println()
    print("       ")
    for j = max(y-offset,1): min(y+offset,size(rslt,2))
        print("---------")
    end
    println()
    for i = max(x-offset,1): min(x+offset,size(rslt,1))
        print(i)
        print("\t|\t")
        for j = max(y-offset,1): min(y+offset,size(rslt,2))
            print(rslt[i, j])
            print("\t")
        end
        println()
    end
    println()
end
# print_error(img2DB, x, y)
print_error(rslt1, x, y)
print_error(rslt2, x, y)

		1	2	3	4	5	6	7	8	9	10	
       ------------------------------------------------------------------------------------------
45	|	0.0	1.0	1.0	1.0	0.0	1.0	1.0	0.0	0.0	0.0	
46	|	0.0	0.0	1.0	1.0	0.0	0.0	1.0	1.0	0.0	0.0	
47	|	1.0	0.0	1.0	1.0	0.0	0.0	1.0	0.0	0.0	0.0	
48	|	1.0	1.0	0.0	0.0	1.0	1.0	0.0	1.0	0.0	1.0	
49	|	0.0	1.0	0.0	1.0	0.0	0.0	1.0	1.0	0.0	0.0	
50	|	0.0	1.0	1.0	0.0	1.0	0.0	1.0	0.0	1.0	1.0	
51	|	1.0	1.0	1.0	0.0	1.0	1.0	1.0	0.0	1.0	0.0	
52	|	0.0	0.0	1.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	
53	|	1.0	1.0	0.0	0.0	0.0	1.0	1.0	1.0	1.0	0.0	
54	|	0.0	1.0	1.0	1.0	1.0	2.0	1.0	0.0	1.0	1.0	
55	|	1.0	1.0	0.0	1.0	0.0	1.0	1.0	0.0	1.0	1.0	

		1	2	3	4	5	6	7	8	9	10	
       ------------------------------------------------------------------------------------------
45	|	8801.0	48801.0	88801.0	128801.0	128802.0	128803.0	128804.0	128805.0	128806.0	128807.0	
46	|	9001.0	49001.0	89001.0	129001.0	129002.0	129003.0	129004.0	129005.0	129006.0	129007.0	
47	|	9201.0	49201.0	89201.0	129201.0	129202.0	129203.0	129204.0	1